In [34]:
import onnxruntime as rt
import numpy as np

onnx_model_v1 = "../data/models/birdnet-onnx-v1.onnx"

test = np.random.randn(5, 144_000).astype(np.float32)

sess = rt.InferenceSession(onnx_model_v1)
y = sess.run(None, {"inputs:0": test})
[res.shape for res in y]

[(320,), (3337,)]

In [14]:
# help(sess)
type(sess)

onnxruntime.capi.onnxruntime_inference_collection.InferenceSession

https://onnxruntime.ai/docs/api/python/api_summary.html#inferencesession

In [12]:
[x.name for x in sess.get_inputs()], [x.name for x in sess.get_outputs()]

(['inputs:0'], ['unknown_242:0', 'unknown_244:0'])

Now, can we programmatically create the model above for unit testing?

In [15]:
144_000 / 3

48000.0

In [29]:
import torch.nn as nn
import torch

# create a super simple model with 2 layers


class MockBirdNET(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(48_000 * 3, 320)
        self.fc2 = nn.Linear(320, 3337)

    def forward(self, x):
        x1 = self.fc1(x)
        x2 = self.fc2(x1)
        return x1, x2


model = MockBirdNET()
model(torch.from_numpy(test))

(tensor([[ 6.5706e-01, -2.3431e-01,  6.7256e-01, -2.3667e-01,  4.4336e-01,
           1.6049e-02,  6.5950e-02, -3.1205e-01,  8.8652e-01, -3.8794e-01,
           1.5483e-01,  9.7137e-01,  1.2336e-01,  5.4181e-01, -6.6283e-01,
           1.9865e-01, -5.6669e-01,  6.7603e-01,  1.8931e-01, -1.5704e+00,
           2.5434e-01,  1.1347e+00,  5.0792e-01,  8.0553e-01,  1.1767e-01,
           4.9501e-01, -5.4933e-01, -4.0768e-01, -2.5366e-01, -1.1838e-01,
           7.2716e-01, -1.0257e-01,  6.7081e-02,  3.7478e-01, -6.2518e-02,
          -2.8208e-02,  3.0539e-01, -3.0376e-01, -7.0898e-01, -1.1575e+00,
           2.8810e-02,  3.7198e-01, -4.2812e-01, -5.5816e-01, -7.4951e-01,
          -6.1835e-02, -2.5026e-01,  1.4488e+00,  3.6471e-01,  1.6374e-01,
          -5.7926e-01, -4.4474e-01, -5.9416e-01,  3.0171e-01,  7.1596e-01,
          -5.5420e-01, -3.1573e-01, -4.7449e-01, -3.2175e-01,  9.3844e-01,
           3.8850e-01,  2.2539e-01,  1.0543e+00,  1.3747e-01,  9.4812e-02,
          -1.0305e+00,  8

In [36]:
# export the model to onxx
from pathlib import Path

test_model_output = "../data/intermediate/test-mock.onnx"
Path(test_model_output).parent.mkdir(exist_ok=True, parents=True)

torch.onnx.export(
    model,
    torch.from_numpy(test),
    test_model_output,
    input_names=["inputs:0"],
    output_names=["unknown_242:0", "unknown_244:0"],
)

mock_model = rt.InferenceSession(test_model_output)
y = mock_model.run(None, {"inputs:0": test})
[res.shape for res in y]

[(5, 320), (5, 3337)]